In [1]:
import warnings
warnings.filterwarnings('ignore')

# About this notebook

* Author: Anubhav Jain
* Github repo: https://github.com/computron/pymatgen_tutorials
* YouTube video: https://youtu.be/2-L6lax1DFU

![alt text](graphics/title.png "Learn Pymatgen Part 4: Structure Transformations, Symmetry Analysis, and Local Environments")

# Structure manipulation and analysis: Transformations, Symmetry, and Local Environment

* In part 1 of this tutorial, we covered how to create basic ``Structure`` objects that represent periodic crystal structures
* This tutorial will cover some more advanced and powerful techniques:
    * Using the **transformations** framework to generate surfaces, doped structures, and more
    * Using the **symmetry** package to analyze the symmetry of ``Structure`` objects
    * Using the **local_env** and **chemenv** modules to analyze local environments around atoms

As in part 1, we will use the ``crystal_toolkit`` library to make the visualizations interactive. Let's get started!  

In [2]:
import crystal_toolkit  # for visualizing structures (install using ``pip install crystal-toolkit``)
import pymatgen  # pymatgen needs to be installed (v2024.6.10 at time of this tutorial)

# Transformations package

The transformations package (``pymatgen.transformations.*``) allows you to make both simple and complex modifications to ``Structure`` objects. For example, pymatgen has implemented transformations that allow you to:
* Perturb structures
* Easily generate surface structures
* Enumerate ordered versions of ``Structure`` objects with site disorder

To use the framework, you first create an instance of the the desire transformation object (with your chosen parameters), and then call ``apply_transformation()`` on your desired structure. We'll cover several examples of this!

In [3]:
# Get a test Structure object that we will transform
from pymatgen.core import Structure, Lattice
structure = Structure.from_file("structures/CsCl.json")
structure

In [4]:
# Use transformation to perturb the atom positions from their original locations
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation

# note the user-adjustable parameters when constructing the transformation object
perturb_transformation = PerturbStructureTransformation(distance=0.6, min_distance=0.4)
perturbed_structure = perturb_transformation.apply_transformation(structure)
perturbed_structure

[2024-10-19 17:11:21,312] ERROR in app: Exception on /_dash-component-suites/dash/dash_table/async-highlight.js [GET]
Traceback (most recent call last):
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore

In [5]:
# Use transformation to create a supercell
from pymatgen.transformations.standard_transformations import SupercellTransformation

supercell_transformation = SupercellTransformation.from_scaling_factors(2, 2, 3)
supercell_structure = supercell_transformation.apply_transformation(perturbed_structure)
supercell_structure


In [6]:
# Get another test structure (LiFePO4 without oxidation states)
structure = Structure.from_file("structures/LiFePO4.json")
print(structure)
structure


Full Formula (Li4 Fe4 P4 O16)
Reduced Formula: LiFePO4
abc   :   4.744800   6.065770  10.410370
angles:  90.501790  90.000190  90.003620
pbc   :       True       True       True
Sites (28)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Li    1e-05    0.99999  0.99999
  1  Li    0.99999  0.5      1e-05
  2  Li    0.49999  0.99999  0.5
  3  Li    0.50002  0.5      0.5
  4  Fe    0.52507  0.25344  0.21884
  5  Fe    0.02508  0.74654  0.28116
  6  Fe    0.97497  0.25346  0.71884
  7  Fe    0.47493  0.74653  0.78116
  8  P     0.58205  0.75169  0.09444
  9  P     0.08207  0.2483   0.40556
 10  P     0.91794  0.75174  0.59443
 11  P     0.41793  0.24828  0.90557
 12  O     0.29156  0.25112  0.04317
 13  O     0.25851  0.75043  0.09622
 14  O     0.71346  0.95594  0.1658
 15  O     0.71627  0.5486   0.16562
 16  O     0.21629  0.4514   0.33438
 17  O     0.21345  0.04406  0.33419
 18  O     0.75852  0.24955  0.40378
 19  O     0.79157  0.7489   0.45682
 20  O  

In [7]:
# Automatically add oxidation states / valences
# Note that the structure is unchanged, the transformation simply adds oxidation state guesses
from pymatgen.transformations.standard_transformations import AutoOxiStateDecorationTransformation

oxi_transformation = AutoOxiStateDecorationTransformation()
oxidized_structure = oxi_transformation.apply_transformation(structure)

# 'oxidized_structure' now has oxidation states / valences assigned
print(oxidized_structure)


Full Formula (Li4 Fe4 P4 O16)
Reduced Formula: LiFePO4
abc   :   4.744800   6.065770  10.410370
angles:  90.501790  90.000190  90.003620
pbc   :       True       True       True
Sites (28)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Li+   1e-05    0.99999  0.99999
  1  Li+   0.99999  0.5      1e-05
  2  Li+   0.49999  0.99999  0.5
  3  Li+   0.50002  0.5      0.5
  4  Fe2+  0.52507  0.25344  0.21884
  5  Fe2+  0.02508  0.74654  0.28116
  6  Fe2+  0.97497  0.25346  0.71884
  7  Fe2+  0.47493  0.74653  0.78116
  8  P5+   0.58205  0.75169  0.09444
  9  P5+   0.08207  0.2483   0.40556
 10  P5+   0.91794  0.75174  0.59443
 11  P5+   0.41793  0.24828  0.90557
 12  O2-   0.29156  0.25112  0.04317
 13  O2-   0.25851  0.75043  0.09622
 14  O2-   0.71346  0.95594  0.1658
 15  O2-   0.71627  0.5486   0.16562
 16  O2-   0.21629  0.4514   0.33438
 17  O2-   0.21345  0.04406  0.33419
 18  O2-   0.75852  0.24955  0.40378
 19  O2-   0.79157  0.7489   0.45682
 20  O2-

In [8]:
# A transformation can perform complex operations
# Next, we will create a surface slab structure with vacuum layers

# Get a test structure (Si)
structure = Structure.from_file("structures/Si.json")
structure

In [9]:
from pymatgen.transformations.advanced_transformations import SlabTransformation

# Define slab parameters
miller_index = [1, 1, 1]  # Miller index for the slab
min_slab_size = 10      # Thickness of the slab
min_vacuum_size = 15    # Thickness of the vacuum layer

# Create the transformation with user parameters
slab_transformation = SlabTransformation(
    miller_index=miller_index,
    min_slab_size=min_slab_size,
    min_vacuum_size=min_vacuum_size,
    lll_reduce=True,  # makes the slab more orthogonal 
    center_slab=True,  # centers the slab with vacuum on top and bottom
    in_unit_planes=True  # means slab thicknesses are specified as number of planes, not Angstroms
)

# Apply the transformation
slab_structure = slab_transformation.apply_transformation(structure)
slab_structure

In [10]:
# Expand the the surface plane (e.g., if you want to add an adsorbate later)
supercell_transformation = SupercellTransformation.from_scaling_factors(2, 2, 1)
supercell_slab_structure = supercell_transformation.apply_transformation(slab_structure)
supercell_slab_structure

## Transformations requiring enumlib

Some transformations may require use of additional libraries for their operation. **To use the following transformations, you need to install the ``enumlib`` library led by Gus Hart.**

Instructions for installing ``enumlib`` can be found at its Github page: https://github.com/msg-byu/enumlib . Overall, you need to make sure that ``enum.x`` and ``makestr.x`` (or ``makestr.py`` for more recent versions) are properly compiled and in your executable PATH.

If using transformations that use ``enumlib``, you should cite the original papers as listed in the enumlib Github repository:

1. Gus L. W. Hart and Rodney W. Forcade, "Algorithm for generating derivative structures," Phys. Rev. B 77 224115, (26 June 2008)

2. Gus L. W. Hart and Rodney W. Forcade, "Generating derivative structures from multilattices: Application to hcp alloys," Phys. Rev. B 80 014120 (July 2009)

3. Gus L. W. Hart, Lance J. Nelson, and Rodney W. Forcade, "Generating derivative structures at a fixed concentration," Comp. Mat. Sci. 59 101-107 (March 2012)

4. Wiley S. Morgan, Gus L. W. Hart, Rodney W. Forcade, "Generating derivative superstructures for systems with high configurational freedom," Comp. Mat. Sci. 136 144-149 (May 2017)

In [11]:
# Get a test structure with site disorder
s_disorder = Structure(Lattice.cubic(4),
                       [{"Li": 0.2, "Na": 0.2, "K": 0.6}, {"O": 1}],
                       [[0, 0, 0], [0.5, 0.5, 0.5]])
print(s_disorder)
s_disorder

Full Formula (K0.6 Na0.2 Li0.2 O1)
Reduced Formula: K0.6Na0.2Li0.2O1
abc   :   4.000000   4.000000   4.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP                       a    b    c
---  ---------------------  ---  ---  ---
  0  K:0.6, Na:0.2, Li:0.2  0    0    0
  1  O                      0.5  0.5  0.5


In [12]:
from pymatgen.transformations.advanced_transformations import EnumerateStructureTransformation

est = EnumerateStructureTransformation(max_cell_size=None, max_disordered_sites=5)
lst_orderings = est.apply_transformation(s_disorder, return_ranked_list=3)  # create 3 orderings

for idx, ostruct in enumerate(lst_orderings):
    print(f"---Ordering {idx+1}")
    print(ostruct)
    print("\n")
    
lst_orderings[0]["structure"]  # visualize the first ordered structure

rm: vasp.{}: No such file or directory


---Ordering 1
{'num_sites': 10, 'structure': Structure Summary
Lattice
    abc : 5.656854249492381 6.928203230275509 8.94427190999916
 angles : 104.96321743330712 71.56505117707799 90.0
 volume : 320.0
      A : 4.0 0.0 4.0
      B : -4.0 4.0 4.0
      C : 0.0 -8.0 4.0
    pbc : True True True
PeriodicSite: K (0.0, 0.0, 8.0) [0.8, 0.8, 0.4]
PeriodicSite: K (0.0, -4.0, 4.0) [0.2, 0.2, 0.6]
PeriodicSite: K (0.0, -4.0, 8.0) [0.6, 0.6, 0.8]
PeriodicSite: Na (0.0, 0.0, 4.0) [0.4, 0.4, 0.2]
PeriodicSite: Li (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: O (2.0, -2.0, 6.0) [0.8, 0.3, 0.4]
PeriodicSite: O (-2.0, -2.0, 6.0) [0.2, 0.7, 0.6]
PeriodicSite: O (2.0, -6.0, 6.0) [0.6, 0.1, 0.8]
PeriodicSite: O (-2.0, 2.0, 2.0) [0.0, 0.5, 0.0]
PeriodicSite: O (-2.0, 2.0, 6.0) [0.4, 0.9, 0.2]}


---Ordering 2
{'num_sites': 10, 'structure': Structure Summary
Lattice
    abc : 5.656854249492381 5.656854249492381 12.0
 angles : 76.3669777746336 90.0 119.99999999999999
 volume : 320.0
      A : 4.0 0.0 4.0
 

In [13]:
# The next example will be a transformation to add a dopant into a structure
# Note that this is a *substitutional* dopant (default is aliovalent substitution)
# It will create a supercell so that the dopant is only a small concentration of the overall cell

# Get a test structure
structure = Structure.from_file("structures/LiFePO4.json")
print(structure)
structure

Full Formula (Li4 Fe4 P4 O16)
Reduced Formula: LiFePO4
abc   :   4.744800   6.065770  10.410370
angles:  90.501790  90.000190  90.003620
pbc   :       True       True       True
Sites (28)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Li    1e-05    0.99999  0.99999
  1  Li    0.99999  0.5      1e-05
  2  Li    0.49999  0.99999  0.5
  3  Li    0.50002  0.5      0.5
  4  Fe    0.52507  0.25344  0.21884
  5  Fe    0.02508  0.74654  0.28116
  6  Fe    0.97497  0.25346  0.71884
  7  Fe    0.47493  0.74653  0.78116
  8  P     0.58205  0.75169  0.09444
  9  P     0.08207  0.2483   0.40556
 10  P     0.91794  0.75174  0.59443
 11  P     0.41793  0.24828  0.90557
 12  O     0.29156  0.25112  0.04317
 13  O     0.25851  0.75043  0.09622
 14  O     0.71346  0.95594  0.1658
 15  O     0.71627  0.5486   0.16562
 16  O     0.21629  0.4514   0.33438
 17  O     0.21345  0.04406  0.33419
 18  O     0.75852  0.24955  0.40378
 19  O     0.79157  0.7489   0.45682
 20  O  

In [14]:
from pymatgen.transformations.advanced_transformations import DopingTransformation

dt = DopingTransformation("Ca2+", min_length=10)
lst_dopings = dt.apply_transformation(structure, return_ranked_list=3)

for idx, ostruct in enumerate(lst_dopings):
    print(f"---Ordering {idx+1}")
    print(ostruct)
    print("\n")

lst_dopings[0]["structure"]

rm: vasp.{}: No such file or directory


---Ordering 1
{'num_sites': 168, 'energy': -6989.243831298399, 'structure': Structure Summary
Lattice
    abc : 10.41037 12.131540000000001 14.234399999999999
 angles : 90.00362 90.00019 90.50178999999999
 volume : 1797.6478062675028
      A : 10.41036999994276 0.0 3.452209424235223e-05
      B : -0.10624530873540061 -12.131074730561894 0.0007664818446308863
      C : 0.0 0.0 -14.234399999999999
    pbc : True True True
PeriodicSite: Li+ (10.36, -6.065, 0.0003703) [1.0, 0.5, 3.333e-06]
PeriodicSite: Li+ (10.3, -12.13, 0.0007536) [1.0, 1.0, 3.333e-06]
PeriodicSite: Li+ (10.36, -6.065, -4.744) [1.0, 0.5, 0.3333]
PeriodicSite: Li+ (10.3, -12.13, -4.744) [1.0, 1.0, 0.3333]
PeriodicSite: Li+ (10.36, -6.065, -9.489) [1.0, 0.5, 0.6667]
PeriodicSite: Li+ (10.3, -12.13, -9.489) [1.0, 1.0, 0.6667]
PeriodicSite: Li+ (-0.02646, -3.033, -4.745) [1e-05, 0.25, 0.3333]
PeriodicSite: Li+ (-0.07958, -9.098, -4.744) [1e-05, 0.75, 0.3333]
PeriodicSite: Li+ (-0.02646, -3.033, -9.489) [1e-05, 0.25, 0.6667]


# Analyzing crystal symmetry using SpaceGroupAnalyzer


Pymatgen can analyze crystal symmetry by making use of the **spglib** library written by Atsushi Togo *et al.* Typically, **spglib** should already be installed automatically when you install **pymatgen**.

If you use the tools in ``SpaceGroupAnalyzer``, you should cite the **spglib** paper:


1. Atsushi Togo, Kohei Shinohara & Isao Tanaka (2024) Spglib: a software
library for crystal symmetry search, Science and Technology of Advanced Materials: Methods,
4:1, 2384822, DOI: 10.1080/27660400.2024.2384822

In [15]:
structure = Structure.from_file("structures/Si.json")
print(structure)
structure

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.840198   3.840199   3.840198
angles: 119.999991  90.000000  60.000009
pbc   :       True       True       True
Sites (2)
  #  SP       a    b     c
---  ----  ----  ---  ----
  0  Si    0     0    0
  1  Si    0.75  0.5  0.75


In [16]:
# getting the space group

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

sga_default = SpacegroupAnalyzer(structure)
sg_symbol, sg_number = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

print(f"Default settings - Structure spacegroup: symbol: {sg_symbol}, number: {sg_number}")

Default settings - Structure spacegroup: symbol: Fd-3m, number: 227


In [17]:
# modifying the numerical tolerances of the SpacegroupAnalyzer

sga_tight = SpacegroupAnalyzer(structure, symprec=1E-5, angle_tolerance=1E-5)  # this is too tight for general use
sg_symbol, sg_number = sga_tight.get_space_group_symbol(), sga_tight.get_space_group_number()
print(f"Tight settings - Structure spacegroup: symbol: {sg_symbol}, number: {sg_number}")

Tight settings - Structure spacegroup: symbol: I4_1/amd, number: 141


In [18]:
perturb_transformation = PerturbStructureTransformation(distance=0.05)

# Apply the transformation
perturbed_structure = perturb_transformation.apply_transformation(structure)
print(perturbed_structure)
perturbed_structure

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.840198   3.840199   3.840198
angles: 119.999991  90.000000  60.000009
pbc   :       True       True       True
Sites (2)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Si    0.98793   0.004919  0.994394
  1  Si    0.755727  0.49869   0.737389


In [19]:
sga_default = SpacegroupAnalyzer(perturbed_structure)
sg_symbol, sg_number = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

print(f"Default settings - Perturbed structure spacegroup: symbol: {sg_symbol}, number: {sg_number}")

Default settings - Perturbed structure spacegroup: symbol: P-1, number: 2


In [20]:
# Getting "ideal" symmetry for a perturbed structure

sga_loose = SpacegroupAnalyzer(perturbed_structure, symprec=0.2, angle_tolerance=10)  # too loose for general use
sg_symbol, sg_number = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()

print(f"Loose settings - Perturbed structure spacegroup: symbol: {sg_symbol}, number: {sg_number}")

Loose settings - Perturbed structure spacegroup: symbol: Fd-3m, number: 227


# Transforming Structures to different settings based on symmetry

Crystal structures can be represented by various *settings*. A particular setting may be optimal for reducing the number of atoms in the unit cell (primitive cell) or for highlighting the symmetry inherent in the structure (conventional cell).

Pymatgen offers tools to get crystal structures in various settings, and also to symmetrize structures in which atoms may not be in their ideal symmetry locations.

In [21]:
structure = Structure.from_file("structures/Si.json")
print(structure)  # primitive cell
structure

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.840198   3.840199   3.840198
angles: 119.999991  90.000000  60.000009
pbc   :       True       True       True
Sites (2)
  #  SP       a    b     c
---  ----  ----  ---  ----
  0  Si    0     0    0
  1  Si    0.75  0.5  0.75


In [22]:
# transform primitive structure to conventional structure representation
conventional_structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()
print(conventional_structure)
conventional_structure

Full Formula (Si8)
Reduced Formula: Si
abc   :   5.430861   5.430861   5.430861
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.75  0.75
  2  Si    0     0.5   0.5
  3  Si    0.25  0.25  0.25
  4  Si    0.5   0     0.5
  5  Si    0.75  0.75  0.25
  6  Si    0.5   0.5   0
  7  Si    0.75  0.25  0.75


In [23]:
# get Wyckoff labels for conventional structure
symmetrized_structure = SpacegroupAnalyzer(conventional_structure).get_symmetrized_structure()
print(symmetrized_structure)
print(symmetrized_structure.equivalent_indices)
print(symmetrized_structure.wyckoff_letters)
print(symmetrized_structure.wyckoff_symbols)


SymmetrizedStructure
Full Formula (Si8)
Reduced Formula: Si
Spacegroup: Fd-3m (227)
abc   :   5.430861   5.430861   5.430861
angles:  90.000000  90.000000  90.000000
Sites (8)
  #  SP      a    b    c  Wyckoff
---  ----  ---  ---  ---  ---------
  0  Si      0    0    0  8a
[[0, 1, 2, 3, 4, 5, 6, 7]]
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
['8a']


In [24]:
# conventional structure back to primitive

# primitive cell standards are in:
#
# Setyawan, W., & Curtarolo, S. (2010). High-throughput electronic band structure 
# calculations: Challenges and tools. Computational Materials Science, 49(2), 299-312.
# doi:10.1016/j.commatsci.2010.05.010.

primitive_structure = SpacegroupAnalyzer(conventional_structure).get_primitive_standard_structure()
print(primitive_structure)
primitive_structure

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.840199   3.840199   3.840199
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25


In [25]:
# perturb the primitive structure and use space group analysis to make the atom positions "perfect" again
perturbed_structure = perturb_transformation.apply_transformation(conventional_structure)
print("---Perturbed structure---")
print(perturbed_structure)

---Perturbed structure---
Full Formula (Si8)
Reduced Formula: Si
abc   :   5.430861   5.430861   5.430861
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Si    0.007489  0.997981  0.995039
  1  Si    0.248985  0.75708   0.744204
  2  Si    0.994407  0.49272   0.500698
  3  Si    0.255197  0.257494  0.251263
  4  Si    0.495836  0.994135  0.505746
  5  Si    0.751543  0.742578  0.255224
  6  Si    0.490899  0.50041   0.00133
  7  Si    0.748475  0.259072  0.750374


In [26]:
sga_loose = SpacegroupAnalyzer(perturbed_structure, symprec=0.2, angle_tolerance=10) 
unperturbed_structure = sga_loose.get_refined_structure()  # space group analyzer with loose tolerances
print("---Refined structure---")
print(unperturbed_structure)

---Refined structure---
Full Formula (Si8)
Reduced Formula: Si
abc   :   5.430861   5.430861   5.430861
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.5   0.5   0
  1  Si    0.25  0.25  0.25
  2  Si    0.5   0     0.5
  3  Si    0.25  0.75  0.75
  4  Si    0     0.5   0.5
  5  Si    0.75  0.25  0.75
  6  Si    0     0     0
  7  Si    0.75  0.75  0.25


# Local environment analysis (coordination number, coordination polyhedra)

In addition to analyzing the *global* symmetry of ``Structure`` objects, **pymatgen** has routines for analyzing *local* environments. There are several algorithms implemented for local environment analysis and this tutorial will demo several of them. We will cover:

* Different methods to assess nearest neighbors and coordination number of sites in a ``Structure``
* Determining the type of coordination polyhedra using an order-parameter based method as well as based on the **Chemenv** package

In [27]:
# Get a test structure
structure = Structure.from_file("structures/LiFePO4.json")
structure

In [28]:
# The MinimumDistanceNN considers neighbors to be (i) the closest atom and 
# (ii) any other atoms that are within a certain distance tolerance of that near neighbor distance
# This is good for simple structures but will fail for complex structures & environments

from pymatgen.analysis.local_env import MinimumDistanceNN
nn_analyzer = MinimumDistanceNN(tol=0.1)  # all neighbors within 10% of minimum distance neighbor
print(f"Min. distance Coordination number of site 0 (Li): {nn_analyzer.get_cn(structure, 0)}")
print(f"Min. distance Coordination number of site 5 (Fe): {nn_analyzer.get_cn(structure, 5)}")
print(f"Min. distance Neighbors of site 5 (Fe):\n {nn_analyzer.get_nn(structure, 5)}")

Min. distance Coordination number of site 0 (Li): 6
Min. distance Coordination number of site 5 (Fe): 5
Min. distance Neighbors of site 5 (Fe):
 [PeriodicNeighbor: O (0.9618, -4.552, -1.226) [0.2585, 0.7504, 0.09622], PeriodicNeighbor: O (1.695, -3.328, 1.346) [-0.2837, 0.5486, 0.1656], PeriodicNeighbor: O (3.424, -6.333, -1.012) [0.2135, 1.044, 0.3342], PeriodicNeighbor: O (4.716, -4.542, 0.9893) [-0.2084, 0.7489, 0.4568], PeriodicNeighbor: O (3.457, -2.738, -1.026) [0.2163, 0.4514, 0.3344]]


In [29]:
# The VoronoiNN algorithm constructs neighbors to be the atoms within the Voronoi polyhedra
# of the central atom.
# It works well for structures with high levels of symmetry, but requires parameter tuning
# for complex structures.

from pymatgen.analysis.local_env import VoronoiNN
nn_analyzer = VoronoiNN()  # note that there are user-settable parameters not shown here
print(f"Voronoi Coordination number of site 5 (Fe): {nn_analyzer.get_cn(structure, 5)}")

Voronoi Coordination number of site 5 (Fe): 15


In [30]:
# The CrystalNN algorithm is tuned to give good "out of the box" results on crystals of various types

# If you use this method, you should cite:
#
# Pan, H.; Ganose, A. M.; Horton, M.; Aykol, M.; Persson, K. A.; Zimmermann, N. E. R.; 
# Jain, A. Benchmarking Coordination Number Prediction Algorithms on Inorganic Crystal 
# Structures. Inorg. Chem. 2021, 60 (3), 1590–1603. https://doi.org/10.1021/acs.inorgchem.0c02996.


from pymatgen.analysis.local_env import CrystalNN
nn_analyzer = CrystalNN()
print(f"CrystalNN Coordination number of site 5 (Fe): {nn_analyzer.get_cn(structure, 5)}")

CrystalNN Coordination number of site 5 (Fe): 6


# The ChemEnv package

The **ChemEnv** package is the most powerful tool in pymatgen for evaluating local environments and coordination polyhedra. Unfortunately, there are an infinite number of user-tunable settings and strategies that are difficult to understand. This tutorial will demo two common "strategies" in **ChemEnv**.

If you use the **ChemEnv** package you should cite the paper below:

1. Waroquiers, D.; George, J.; Horton, M.; Schenk, S.; Persson, K. A.; Rignanese, G.-M.; Gonze, X.; Hautier, G. ChemEnv : A Fast and Robust Coordination Environment Identification Tool. Acta Crystallogr B Struct Sci Cryst Eng Mater 2020, 76 (4), 683–695. https://doi.org/10.1107/S2052520620007994.

The paper also links to another tutorial for **ChemEnv**: https://journals.iucr.org/b/issues/2020/04/00/lo5066/lo5066sup1.pdf


In [31]:
structure = Structure.from_file("structures/LiFePO4.json")
structure

In [32]:
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder

# determine the coordination environments - this does the basic pre-processing needed by ChemEnv
# It tries to make a "map" of different possible coordination environments, which can subsequently be
# processed to give probabilities of different environments

lgf = LocalGeometryFinder()
lgf.setup_structure(structure)
se = lgf.compute_structure_environments(maximum_distance_factor=1.41)


In [33]:
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy

# ChemEnv has multiple "strategies" for giving answers, and multiple levels of detail in which to report results
# In the simplest "strategy" and level of detail, each site in the structure has a single coordination environment
strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)
lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)


# print the coordination environments
# O:6=octahedral, T:4=tetrahedral, S:1=single neighbor, A:2=angular, TY:3=Triangular non-coplanar
# A description of all the coordination environments is in: 
# https://journals.iucr.org/b/issues/2020/04/00/lo5066/lo5066sup3.pdf
for i in range(len(structure.sites)):
    print(f"Site: {structure.sites[i]}\n Coordination: {lse.coordination_environments[i]}\n\n")

Site: [ 1.03571438e+01 -6.06547671e+00  3.70310839e-04] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.074975444793722, 'permutation': [0, 3, 5, 1, 4, 2]}]


Site: [-0.02645722 -3.03276868 -4.74456093] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.045412985844226, 'permutation': [3, 4, 5, 0, 2, 1]}]


Site: [ 5.15206288 -6.06547671 -2.37195205] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.0750553403164074, 'permutation': [2, 4, 5, 1, 0, 3]}]


Site: [ 5.17862367 -3.03276868 -2.37228601] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.0443429346309263, 'permutation': [3, 5, 4, 2, 0, 1]}]


Site: [ 2.26474197 -1.53724979 -2.49124745] Fe
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.3687267573000335, 'permutation': [1, 4, 3, 2, 5, 0]}]


Site: [ 2.88732144 -4.52816626 -0.11870377] Fe
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1, 'csm': 2.36886727996453, 'permutation': [0, 4, 2, 5, 

In [34]:
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import MultiWeightsChemenvStrategy

# Another "strategy" is the MultiWeights strategy in which ChemEnv provides a probabilistic assessment
# of different coordination envirments
strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()

lse = LightStructureEnvironments.from_structure_environments(
    strategy=strategy, structure_environments=se
)

# print the coordination environments
# O:6=octahedral, T:4=tetrahedral, S:1=single neighbor, A:2=angular, TY:3=Triangular non-coplanar
# SS:4=see-saw, SY:4=square non-planar
# A description of all the coordination environments is in: 
# https://journals.iucr.org/b/issues/2020/04/00/lo5066/lo5066sup3.pdf
for i in range(len(structure.sites)):
    print(f"Site: {structure.sites[i]}\n Coordination: {lse.coordination_environments[i]}\n\n")

Site: [ 1.03571438e+01 -6.06547671e+00  3.70310839e-04] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1.0, 'csm': 2.074975444793722, 'permutation': [0, 3, 5, 1, 4, 2]}]


Site: [-0.02645722 -3.03276868 -4.74456093] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1.0, 'csm': 2.045412985844226, 'permutation': [3, 4, 5, 0, 2, 1]}]


Site: [ 5.15206288 -6.06547671 -2.37195205] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1.0, 'csm': 2.0750553403164074, 'permutation': [2, 4, 5, 1, 0, 3]}]


Site: [ 5.17862367 -3.03276868 -2.37228601] Li
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 1.0, 'csm': 2.0443429346309263, 'permutation': [3, 5, 4, 2, 0, 1]}]


Site: [ 2.26474197 -1.53724979 -2.49124745] Fe
 Coordination: [{'ce_symbol': 'O:6', 'ce_fraction': 0.6592417517712533, 'csm': 2.3687267573000335, 'permutation': [1, 4, 3, 2, 5, 0]}, {'ce_symbol': 'SS:4', 'ce_fraction': 0.3407582482205001, 'csm': 0.07964041379461761, 'permutation': [0, 1, 2, 3]}, {'ce_symbol': 'SY